In [1]:
import tensorflow as tf
#tf.enable_eager_execution()

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

import json

RESULTS_FILE = 'results.json'


In [4]:
import os
os.environ['AUTOGRAPH_VERBOSITY'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
config = _config.build_config()
config['use_tpu'] = False
config['train_data_files'] = ['../data/dataset/train_1_KD.tfrecords', '../data/dataset/train_2_KD.tfrecords', '../data/dataset/train_3_KD.tfrecords', '../data/dataset/train_4_KD.tfrecords']
config['valid_data_file'] = ['../data/dataset/validation_KD.tfrecords']
config['test_data_file'] = ['../data/dataset/test_KD.tfrecords']
config['sample_data_file'] = ['../data/dataset/sample_KD.tfrecords']
config['train_epochs'] = 10

config['use_KD'] = True
config['imitation_lmb'] = 0.7
config['temperature'] = 4.0

# for files_group in ['train_data_files', 'valid_data_file', 'test_data_file', 'sample_data_file']:
#     for i in range(len(config[files_group])):
#         config[files_group][i] = config[files_group][i].replace('.', '_KD.')

if not config['use_KD']:
    for i in range(len(config['train_data_files'])):
        config['train_data_files'][i] = config['train_data_files'][i].replace("_KD", "")
    config['valid_data_file'][0] = config['valid_data_file'][0].replace("_KD", "")
    config['test_data_file'][0] = config['test_data_file'][0].replace("_KD", "")
    config['sample_data_file'][0] = config['sample_data_file'][0].replace("_KD", "")



matrix=[[0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0]] 
# Operations at the vertices of the module, matches order of matrix
labels=['input',
        'conv1x1-bn-relu',
        'conv1x1-bn-relu',
        'conv1x1-bn-relu',
        'conv1x1-bn-relu',
        'output']

matrix = np.array(matrix)
labels = np.array(labels)

spec = model_spec.ModelSpec(matrix, labels)
model_dir = '../data/donor_data'

In [6]:
from nasbench.lib.evaluate import _TrainAndEvaluator

In [7]:
evaluator = _TrainAndEvaluator(spec, config, './tmp')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': './tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 7490, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_s

In [8]:
meta = evaluator.run()







--------------------------------------------------->
next_evaluation:  0.5
self.config['train_epochs']:  10
epoch:  5.0
self.input_train.num_images:  40000
self.config['batch_size']:  256
train_steps:  781
-------------------------------------------------- 


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7fc384516f28>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

Converted call: <function _create_dataset_reader.<locals>.read_one_file at 0x7fc384516f28>
    args: (<tf.Tensor 'args_0:0' shape=() dtype=string>,)
    kwargs: {}

INFO:tensorflow:Whitelisted: <function _create_dataset_reader.<locals>.read_one_file at 0x7fc384516f28>: DoNotConvert rule for tensorflow
Whitelisted: <func

In [9]:
[{k:v for k, v in res.items() if k != 'sample_metrics'} for res in meta['evaluation_results'] ]

[{'epochs': 5.0,
  'training_time': 61.941,
  'training_steps': 781,
  'train_accuracy': 0.17407852,
  'validation_accuracy': 0.16947116,
  'test_accuracy': 0.16997196,
  'predict_time': 5.573829174041748},
 {'epochs': 10.0,
  'training_time': 127.19,
  'training_steps': 1562,
  'train_accuracy': 0.38161057,
  'validation_accuracy': 0.37129408,
  'test_accuracy': 0.37409857,
  'predict_time': 5.521076917648315}]

KD   , RMSProp, 0.28796074
NO KD, RMSProp, 0.3466546
KD,    Adam,    0.37129408
NO KD  Adam,    0.33663863



In [12]:
input_sample = cifar.CIFARInput('sample', config)

In [13]:
dataset = input_sample.input_fn({'batch_size':100, 'use_KD':config['use_KD']})

In [14]:
dataset

<PrefetchDataset shapes: ((100, 32, 32, 3), (100, 11)), types: (tf.float32, tf.float32)>

In [10]:
dataset = tf.data.TFRecordDataset(config['sample_data_file'])

In [15]:
for element in dataset: 
    #print(element)
    break

In [16]:
element

(<tf.Tensor: id=87, shape=(100, 32, 32, 3), dtype=float32, numpy=
 array([[[[ 1.7890141 ,  1.9495894 ,  2.1008997 ],
          [ 1.9477695 ,  2.0784345 ,  2.1158922 ],
          [ 2.027147  ,  2.0784345 ,  2.0859072 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ]],
 
         [[ 0.05858076,  0.27460143,  0.9314843 ],
          [ 0.51897126,  0.74166536,  1.2013494 ],
          [ 1.8525163 ,  1.965695  ,  2.0559223 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ]],
 
         [[-0.7986981 , -0.49846992,  0.4067466 ],
          [-0.5923162 , -0.2568851 ,  0.54167914],
          [ 0.8841086 ,  1.128201  ,  1.5611695 ],
          ...,
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.0588982 ,  2.1267514 ,  2.1158922 ],
          [ 2.05

In [17]:
element[1]

<tf.Tensor: id=88, shape=(100, 11), dtype=float32, numpy=
array([[ 0.        , 27.231663  ,  2.5751529 , ..., -4.6600294 ,
         2.0751333 ,  2.8788328 ],
       [ 0.        , 16.118654  , -2.0107608 , ..., -7.676149  ,
         4.625721  , -3.3237    ],
       [ 0.        , 13.896395  , -3.0747104 , ..., -2.1238356 ,
        -7.08816   ,  4.364662  ],
       ...,
       [ 9.        ,  5.706644  ,  9.426671  , ..., -4.891599  ,
        -2.3245106 , 30.134348  ],
       [ 9.        , -1.8154393 ,  4.420699  , ..., -3.022294  ,
        -4.9302526 , 30.931654  ],
       [ 9.        ,  0.9914434 , 10.077514  , ..., -0.89257365,
         6.685395  , 15.760097  ]], dtype=float32)>

In [15]:
features = tf.parse_single_example(
      element,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([11], tf.float32),
      })

In [16]:
features

{'image': <tf.Tensor: id=50, shape=(), dtype=string, numpy=b'\xee\xf8\xfd\xfd\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x81\x9e\xf2\xfe\xff\xfc\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffKX\xb5\xfd\xff\xfc\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffEEl\xe5\xfe\xfe\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffDAQ\x9c\xf8\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffH?Ob\xd0\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffQ@NS\x88\xf2\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff_APRU\xba\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x

In [20]:
from tensorflow.estimator import ModeKeys

In [21]:
features, labels, input_hooks = (
          evaluator.estimator._get_features_and_labels_from_input_fn(
input_sample.input_fn, ModeKeys.TRAIN))

KeyError: 'use_KD'

In [28]:
features

<tf.Tensor 'IteratorGetNext_1:0' shape=(256, 32, 32, 3) dtype=float32>

In [29]:
labels

<tf.Tensor 'IteratorGetNext_1:1' shape=(256,) dtype=int32>

In [30]:
config

{'train_data_files': ['train_1_KD.tfrecords',
  'train_2_KD.tfrecords',
  'train_3_KD.tfrecords',
  'train_4_KD.tfrecords'],
 'valid_data_file': ['validation_KD.tfrecords'],
 'test_data_file': ['test_KD.tfrecords'],
 'sample_data_file': ['sample_KD.tfrecords'],
 'data_format': 'channels_last',
 'num_labels': 10,
 'module_vertices': 7,
 'max_edges': 9,
 'available_ops': ['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'],
 'stem_filter_size': 128,
 'num_stacks': 3,
 'num_modules_per_stack': 3,
 'batch_size': 256,
 'train_epochs': 36,
 'train_seconds': 14400.0,
 'learning_rate': 0.1,
 'lr_decay_method': 'COSINE_BY_STEP',
 'momentum': 0.9,
 'weight_decay': 0.0001,
 'max_attempts': 5,
 'intermediate_evaluations': ['0.5'],
 'num_repeats': 3,
 'use_tpu': False,
 'tpu_iterations_per_loop': 100,
 'tpu_num_shards': 2,
 'use_KD': True,
 'imitation_lmb': 0.7,
 'temperature': 20.0}